My attempt at understanding the notebook from : https://www.kaggle.com/muellerzr/end-to-end-fastai2

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip


In [2]:
!pip install fastai --quiet

# its no longer fastai2

In [3]:
from fastai.text.all import *

In [4]:
path = Path('../input/jigsaw-toxic-comment-classification-challenge')

### Unzip everything

In [5]:
from zipfile import ZipFile

with ZipFile(path/'train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working')

with ZipFile(path/'test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working')
    
with ZipFile(path/'test_labels.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working')
    
with ZipFile(path/'sample_submission.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('../output/kaggle/working')

In [6]:
path_w = Path('../output/kaggle/working')

In [7]:
path_w.ls()

(#4) [Path('../output/kaggle/working/train.csv'),Path('../output/kaggle/working/test_labels.csv'),Path('../output/kaggle/working/sample_submission.csv'),Path('../output/kaggle/working/test.csv')]

lets us see our training data

In [8]:
df = pd.read_csv(path_w/'train.csv')



In [9]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when your view completely contradicts the coverage in reliable sources, why should anyone care what you feel? You can't even give a consistent argument - is the opening only supposed to mention significant aspects, or the """"most significant"""" ones? \n\n""",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is a horrible thing you put on my talk page. 128.61.19.93,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for prostitution ring. - Crunch Captain.",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand. I came here and my idea was bad right away. What kind of community goes """"you have bad ideas"""" go away, instead of helping rewrite them. """,0,0,0,0,0,0


### Creating tokens

creating a textblock for classification: 

In [10]:
blocks = (TextBlock.from_df(text_cols='comment_text', is_lm=True, res_col_name='text'))

# creates tokenized output

### looking at test data

In [11]:
test_df = pd.read_csv(path_w/'test.csv')

In [12]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"
1,0000247867823ef7,"== From RfC == \n\n The title is fine as it is, IMO."
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lapland — / """
3,00017563c3f7919a,":If you have a look back at the source, the information I updated was the correct form. I can only guess the source hadn't updated. I shall update the information once again but thank you for your message."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [13]:
text_df = pd.Series.append(df['comment_text'], test_df['comment_text'])

In [14]:
text_df = pd.DataFrame(text_df)

In [15]:
text_df.head()

,comment_text
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)"
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_ar..."
4,"You, sir, are my hero. Any chance you remember what page that's on?"


Now you may notice there was a res_col_name parameter. This is where our tokenized text will be output to. Next we need to tell how to grab our x and how we want to split our data. We'll split by 10% randomly:

In [16]:
# grabbing tokenised output
get_x = ColReader('text')

splitter = RandomSplitter(0.1, seed=42)

### creating the datablock

In [17]:
lm_dblock = DataBlock(blocks=blocks,
                      get_x = get_x,
                      splitter = splitter)

In [18]:
lm_dls = lm_dblock.dataloaders(text_df, bs=64)

### creating the language learner

In [19]:
lm_learn = language_model_learner(lm_dls, AWD_LSTM,pretrained=True, metrics=[accuracy, Perplexity()])

In [ ]:
# mixed precision training
# https://docs.fast.ai/callback.fp16

lm_learn.to_fp16()


lm_learn.fine_tune(10,4e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
lm_learn.save_encoder('fine_tuned')

# creating a classifier

In [ ]:
ys = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate']


### Creating Token

here we are using seq_len and vocab from the language model

In [ ]:
blocks = (TextBlock.from_df('comment_text', seq_len=lm_dls.seq_len, vocab=lm_dls.vocab), 
          MultiCategoryBlock(encoded=True, vocab=ys))


### creating datablock

In [ ]:
toxic_clas = DataBlock(blocks=blocks,
                       get_x = ColReader('text'),
                       get_y = ColReader(ys),
                       splitter = RandomSplitter())

In [ ]:
toxic_clas.summary(df.iloc[:100])

### dataloaders from datablock

In [ ]:
dls =toxic_clas.dataloaders(df)

### Defining loss function and metrics

In [ ]:
loss_func = BCEWithLogitsLossFlat(thresh=0.8)
metrics = [partial(accuracy_multi, thresh=0.8)]

### our learner

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=metrics, loss_func=loss_func)

### finding the optimum learning rate

In [ ]:
learn.lr_find()

In [ ]:
learn.load_encoder('fine_tuned');

In [ ]:
learn.to_fp16()

lr = 1e-2
moms = (0.8,0.7, 0.8)
lr *= learn.dls.bs/128
learn.fit_one_cycle(1, lr, moms=moms, wd=0.1)

### unfreezing layer to finetune

In [ ]:
learn.freeze_to(-2)
lr/=2
learn.fit_one_cycle(1, slice(lr/(2.6**4), lr), moms=moms, wd=0.1)

In [ ]:
learn.freeze_to(-3)
lr /=2
learn.fit_one_cycle(1, slice(lr/(2.6**4), lr), moms=moms, wd=0.1)

In [ ]:
learn.unfreeze()
lr /= 5
learn.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7,0.8), wd=0.1)

# submitting the predictions

In [ ]:
dl = learn.dls.test_dl(test_df['comment_text'])

### inference

In [ ]:
preds = learn.get_preds(dl=dl)

In [ ]:
sub = pd.read_csv(path_w/'sample_submission.csv')

In [ ]:
sub.head()

In [ ]:
preds[0][0].cpu().numpy()

# what is this

In [ ]:
sub[ys] = preds[0]

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)